In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import pdfplumber
import re
import zipfile
import tempfile
import os
from io import BytesIO
from sentence_transformers import SentenceTransformer, util
from sklearn.manifold import TSNE

st.set_page_config(layout="wide")
st.title("📂📑 Análise de Ementas de Psicologia via ZIP")

# 1) Upload do ZIP
uploaded_zip = st.file_uploader(
    "Faça upload do arquivo ZIP com todos os PDFs de ementas",
    type=["zip"]
)

if not uploaded_zip:
    st.info("Aguardando upload do ZIP...")
    st.stop()

# 2) Extrai em pasta temporária
with tempfile.TemporaryDirectory() as tmpdir:
    z = zipfile.ZipFile(uploaded_zip)
    z.extractall(tmpdir)

    # 3) Varre todos os PDFs
    registros = []
    for root, _, files in os.walk(tmpdir):
        for fn in files:
            if fn.lower().endswith(".pdf"):
                path = os.path.join(root, fn)
                # extrai texto
                texto = ""
                with pdfplumber.open(path) as pdf:
                    for p in pdf.pages:
                        txt = p.extract_text() or ""
                        texto += txt + "\n"
                # regex para nome e código
                m = re.search(r"UNIDADE CURRICULAR[:\s]*(.+?)\s*\(\s*(\d+)\s*\)", texto, re.IGNORECASE|re.DOTALL)
                nome = m.group(1).strip() if m else fn
                cod  = m.group(2).strip() if m else fn
                # regex para conteúdo programático
                m2 = re.search(
                    r"Conte[úu]do program[aá]tico\s*[:\-–]?\s*(.*?)\s*(?:\n\s*Bibliografia|\Z)",
                    texto, re.IGNORECASE|re.DOTALL
                )
                conteudo = m2.group(1).strip() if m2 else ""
                registros.append({"COD_EMENTA": cod,
                                  "NOME UC": nome,
                                  "CONTEUDO_PROGRAMATICO": conteudo})
    df_ementas = pd.DataFrame(registros)

st.success(f"{len(df_ementas)} ementas carregadas.")

# Botão de preview
st.subheader("Preview das primeiras ementas")
st.dataframe(df_ementas.head(5))

# Carrega ENADE
uploaded_enade = st.file_uploader(
    "Faça upload do Excel de competências ENADE",
    type=["xlsx"], key="enade"
)
if not uploaded_enade:
    st.info("Envie o arquivo ENADE para prosseguir.")
    st.stop()

enade = pd.read_excel(uploaded_enade).dropna(subset=['DESCRIÇÃO'])

# Explode frases ENADE
enade['FRASE_ENADE'] = (enade['DESCRIÇÃO']
                        .str.replace('\n',' ')
                        .str.split(r'[.;]')).explode('DESCRIÇÃO')
enade_expl = enade.assign(FRASE_ENADE=lambda df: df['DESCRIÇÃO'].str.strip())
enade_expl = enade_expl[enade_expl['FRASE_ENADE'].str.len()>5]

# Selecione análise
analise = st.sidebar.selectbox("Escolha a Análise", [
    "t-SNE das UCs",
    "Matriz de Similaridade ENADE × Ementas",
    "Matriz de Redundância"
])

# Pré-carrega modelo
@st.cache_resource
def load_model():
    return SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

model = load_model()

# t-SNE
if analise == "t-SNE das UCs":
    st.header("t-SNE das UCs")
    df_group = (df_ementas.groupby(['COD_EMENTA','NOME UC'])['CONTEUDO_PROGRAMATICO']
                          .apply(lambda texts: " ".join(texts)).reset_index())
    texts = df_group['CONTEUDO_PROGRAMATICO'].tolist()
    emb = model.encode(texts, convert_to_tensor=False)
    n = len(texts)
    perp = st.slider("Perplexity", 2, max(2,n//3), value=min(30,max(2,n//3)))
    tsne = TSNE(n_components=2, random_state=42, perplexity=perp)
    coords = tsne.fit_transform(emb)
    df_group['X'], df_group['Y'] = coords[:,0], coords[:,1]
    fig, ax = st.pyplot() if False else plt.subplots(figsize=(8,6))
    labels = df_group['NOME UC'].unique()
    cmap = plt.cm.get_cmap('tab20', len(labels))
    for i,l in enumerate(labels):
        sub = df_group[df_group['NOME UC']==l]
        ax.scatter(sub['X'], sub['Y'], label=l, color=cmap(i), s=40, alpha=0.7)
    ax.legend(bbox_to_anchor=(1,1)); ax.set_xlabel("t-SNE 1"); ax.set_ylabel("t-SNE 2")
    st.pyplot(fig)

# Matriz de Similaridade
elif analise == "Matriz de Similaridade ENADE × Ementas":
    st.header("Similaridade ENADE × Ementas")
    # explode ementas
    ementa_expl = df_ementas.assign(
        FRASE=lambda df: df['CONTEUDO_PROGRAMATICO']
            .str.replace('\n',' ')
            .str.split(r'[.;]')
    ).explode('FRASE').assign(FRASE=lambda df: df['FRASE'].str.strip())
    ementa_expl = ementa_expl[ementa_expl['FRASE'].str.len()>5]
    with st.spinner("Gerando embeddings…"):
        emb_e = model.encode(ementa_expl['FRASE'].tolist(), convert_to_tensor=True)
        emb_n = model.encode(enade_expl['FRASE_ENADE'].tolist(), convert_to_tensor=True)
    sim = util.cos_sim(emb_n, emb_e).cpu().numpy()
    # coleta max por combinação
    rec = []
    idxs = ementa_expl.groupby('COD_EMENTA').indices
    for cod, sidx in idxs.items():
        for i,row in enade_expl.iterrows():
            subsim = sim[i, sidx]
            rec.append({
                "COD_EMENTA": cod,
                "FRASE_ENADE": row['FRASE_ENADE'],
                "MAX_SIM": float(subsim.max())
            })
    df_sim = pd.DataFrame(rec).pivot(index='COD_EMENTA', columns='FRASE_ENADE', values='MAX_SIM').fillna(0)
    st.dataframe(df_sim.style.background_gradient(cmap="RdYlGn_r"))
    toexcel = BytesIO(); df_sim.to_excel(toexcel); toexcel.seek(0)
    st.download_button("⬇️ Baixar Similaridade", toexcel, "sim_enade_ementa.xlsx")

# Matriz de Redundância
else:
    st.header("Matriz de Redundância")
    df_group = (df_ementas.groupby(['COD_EMENTA'])['CONTEUDO_PROGRAMATICO']
                         .apply(lambda txts: " ".join(txts)).reset_index())
    emb = model.encode(df_group['CONTEUDO_PROGRAMATICO'].tolist(), convert_to_tensor=True)
    sim = util.cos_sim(emb, emb).cpu().numpy()
    df_red = pd.DataFrame(sim, index=df_group['COD_EMENTA'], columns=df_group['COD_EMENTA'])
    st.dataframe(df_red.style.background_gradient(cmap="RdYlGn_r"))
    toexcel = BytesIO(); df_red.to_excel(toexcel); toexcel.seek(0)
    st.download_button("⬇️ Baixar Redundância", toexcel, "redundancia_uc.xlsx")